In [23]:
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
import requests
from datetime import datetime
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely import to_geojson
import os

%matplotlib inline

In [3]:
def create_mds_data(df, G):
    df.sort_values('timestamp', inplace=True, ignore_index=True)

    df['lat'] = df['lat'].apply(lambda x: round(x,3))
    df['lon'] = df['lon'].apply(lambda x: round(x,3))
    reduced_df = df.drop_duplicates(['bike_id','lat', 'lon'])
    point_counts = reduced_df['bike_id'].value_counts()
    ids = point_counts[point_counts > 1].keys()

    reduced_df = reduced_df[reduced_df['bike_id'].isin(ids)].reset_index(drop=True)
    reduced_df['timestamp'] = pd.to_datetime(reduced_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')


    dates, bike_ids, edges, lengths, global_edges = [], [], [], [], []

    for id in ids:
        trip = reduced_df[reduced_df['bike_id'] == id].reset_index(drop=True)
        for i in range(1,len(trip.index)):
            edges_i = []
            length = 0
            start = ox.nearest_nodes(G, trip['lon'][i-1], trip['lat'][i-1])
            end = ox.nearest_nodes(G, trip['lon'][i], trip['lat'][i])
            path = ox.shortest_path(G, start, end)
            try:
                for j in range(1,len(path)):
                    length += G.get_edge_data(path[j-1],path[j])[0]['length']
                    edges_i.append((path[j-1],path[j]))
                    global_edges.append((path[j-1],path[j]))
                dates.append(trip['timestamp'][0].date())
                bike_ids.append(id)
                edges.append(edges_i)
                lengths.append(length)
            except:
                pass

    trip_df = pd.DataFrame({'date':dates, 'bike_id':bike_ids, 'trip_edges':edges, 'trip_length':lengths})

    

    return trip_df, global_edges

In [4]:
def get_name_shape(G:nx.classes.multidigraph.MultiDiGraph) -> pd.DataFrame:
    nodes = ox.graph_to_gdfs(G, edges=False)
    nodes_dict = nodes.to_dict()

    G_edges = list(G.edges())
    G_shapes = []
    G_names = []
    for edge in G_edges:
        try:
            name_i = G.get_edge_data(edge[0],edge[1])[0]['name']
            G_names.append(name_i)
        except:
            G_names.append(None)
        try:
            shape_i = G.get_edge_data(edge[0],edge[1])[0]['geometry']
            G_shapes.append(shape_i)
        except:
            start = (nodes_dict['x'][edge[0]],nodes_dict['y'][edge[0]])
            end = (nodes_dict['x'][edge[1]],nodes_dict['y'][edge[1]])
            G_shapes.append(LineString([start,end]))

    #G_edges = [str(edge) for edge in G_edges]
    #G_shapes = [str(edge) for edge in G_shapes]
    #G_names = [str(edge) for edge in G_names]

    name_shape_df = pd.DataFrame({'edge':G_edges, 'name':G_names, 'shape':G_shapes})
    #edge_name_dict = dict(zip(G_edges,G_names))
    #edge_shape_dict = dict(zip(G_edges,G_shapes))
    return name_shape_df

In [5]:
G = ox.graph_from_place('Prague, Czechia', network_type='drive')

In [6]:
name_shape_df = get_name_shape(G)

In [20]:
import sys
sys.path.append('../')

In [21]:
user = os.environ.get("oup_usr")
pw = os.environ.get("oup_pw")

In [22]:
print(user)

None


In [12]:
url = 'https://pilot.dksr.city/UrbanPulseData/historic/sensordata?eventTypeName=NextBikePragV1EventType&since=2024-09-01T00:00:00.000Z&until=2024-04-03T00:00:00.000Z'
text = requests.get(url, auth=(user, pw))

data = json.loads(text.text)
df = pd.json_normalize(data['sensordata'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
text.text

'Invalid credentials.'